Script to load model and perform predictions on new dataset

In [36]:
import joblib
import pandas as pd
import datetime
import numpy as np
import mysql.connector
loaded_model = joblib.load('fullnesspredictions_model_Save1.sav')

Load the new dataset and clean (for now, this is the original)

In [46]:
df = pd.read_csv('July_15_90MinCutoff_Final_WithID.csv')
youtube = pd.read_csv('YoutubeViews.csv')

mydb = mysql.connector.connect(
  host="cinesave-2.chxybcbl9we4.us-east-1.rds.amazonaws.com",
  user="admin",
  password="Cinesave1",
  database = "sample"
)

print(mydb)
mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM movie_revenue_third")

myresult = mycursor.fetchall()
external = pd.DataFrame(myresult, columns = mycursor.column_names)

youtube = youtube.drop(columns = ['Unnamed: 0'])
external['name'] = external['name'].str.replace('\n', '')
external['in_collection'] = np.where(external['collection'] == '', 0, 1)
external = external.drop(columns = ['collection', 'budget', 'genre', 'popularity', 'release_date', 'revenue', 'production_company', 'earnings'])
external.columns = ['Title', 'runtime', 'vote_average', 'vote_count', 'in_collection']
df = pd.merge(df, external, on = 'Title', how = 'left')
df = df.drop(columns = ['vote_average'])
df['runtime'] = df['runtime'].fillna(value = 132)
df['in_collection'] = df['in_collection'].fillna(value = 0)
df['vote_count'] = df['vote_count'].fillna(value = 7391)
youtube['movie'] = youtube['movie'].str.replace('\n', '')
youtube['trailer views'] = youtube['trailer views'].str.replace(' views', '')
youtube['trailer views'] = youtube['trailer views'].str.replace(',', '')
youtube['trailer views'] = youtube['trailer views'].astype(int)
youtube.columns = ['Title', 'trailer_views']
df_new = pd.merge(df, youtube, how = 'left', on = 'Title')
df_na = df_new[df_new['trailer_views'].isna() == True]
df_new = df_new.dropna()
df = df_new
df = df.loc[df['Time Difference'] >= 0]
df = df.loc[df['Previous_Fullness'] != 0]
df = df.drop(columns = ['Show Date', 'Release Date', 'theater identifier', 'release date', 'Show_Datetime'])
df['vote_daysout_ratio'] = df['vote_count'] / df['days_out']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
id_list = df['id']
X = df.drop(columns = ['id','Title', 'Actual_Fullness', 'vote_daysout_ratio'])
X[X.columns] = scaler.fit_transform(X[X.columns])



In [47]:
prices = df[['id', 'price']]
prices


,id,price
10,41,16.49
11,62,16.49
12,65,16.49
13,85,16.49
14,103,16.49
...,...,...
1041,4203,16.49
1042,4234,16.49
1043,4241,16.49
1044,4247,16.49


Use the model on the new data

In [48]:
X['pred'] = loaded_model.predict(X)
X

,price,time,Time Taken,Time Difference,Earnings,IMDB rating,Popularity,Star Power,days_out,Week_Day,Weekend,Night,Previous_Fullness,runtime,vote_count,in_collection,trailer_views,pred
10,0.487179,0.306122,0.285521,0.400000,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,0.0,0.124640,0.673469,0.816827,0.0,0.348484,0.351843
11,0.487179,0.551020,0.484438,0.766667,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,0.0,0.124640,0.673469,0.816827,0.0,0.348484,0.464426
12,0.487179,0.306122,0.322057,0.100000,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,0.0,0.124640,0.673469,0.816827,0.0,0.348484,0.401636
13,0.487179,0.551020,0.549391,0.233333,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,0.0,0.124640,0.673469,0.816827,0.0,0.348484,0.554965
14,0.487179,0.795918,0.714479,0.877778,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,1.0,0.124640,0.673469,0.816827,0.0,0.348484,0.619917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,0.487179,0.469388,0.389716,0.877778,0.005398,0.52381,0.106595,0.253357,0.679825,0.0,0.0,0.0,0.253368,0.448980,0.048503,0.0,0.101945,0.160437
1042,0.487179,0.469388,0.389716,0.877778,0.005398,0.52381,0.106595,0.253357,0.679825,0.0,0.0,0.0,0.253368,0.448980,0.048503,0.0,0.101945,0.160437
1043,0.487179,0.469388,0.465494,0.255556,0.005398,0.52381,0.106595,0.253357,0.679825,0.0,0.0,0.0,0.253368,0.448980,0.048503,0.0,0.101945,0.173627
1044,0.487179,0.469388,0.427605,0.566667,0.005398,0.52381,0.106595,0.253357,0.679825,0.0,0.0,0.0,0.253368,0.448980,0.048503,0.0,0.101945,0.172797


Put new values in the database

In [49]:
id_list

10        41
11        62
12        65
13        85
14       103
        ... 
1041    4203
1042    4234
1043    4241
1044    4247
1045    4250
Name: id, Length: 580, dtype: int64

In [50]:
X['id'] = id_list
X

,price,time,Time Taken,Time Difference,Earnings,IMDB rating,Popularity,Star Power,days_out,Week_Day,Weekend,Night,Previous_Fullness,runtime,vote_count,in_collection,trailer_views,pred,id
10,0.487179,0.306122,0.285521,0.400000,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,0.0,0.124640,0.673469,0.816827,0.0,0.348484,0.351843,41
11,0.487179,0.551020,0.484438,0.766667,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,0.0,0.124640,0.673469,0.816827,0.0,0.348484,0.464426,62
12,0.487179,0.306122,0.322057,0.100000,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,0.0,0.124640,0.673469,0.816827,0.0,0.348484,0.401636,65
13,0.487179,0.551020,0.549391,0.233333,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,0.0,0.124640,0.673469,0.816827,0.0,0.348484,0.554965,85
14,0.487179,0.795918,0.714479,0.877778,0.478079,0.97619,0.001150,0.241676,0.701754,1.0,1.0,1.0,0.124640,0.673469,0.816827,0.0,0.348484,0.619917,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,0.487179,0.469388,0.389716,0.877778,0.005398,0.52381,0.106595,0.253357,0.679825,0.0,0.0,0.0,0.253368,0.448980,0.048503,0.0,0.101945,0.160437,4203
1042,0.487179,0.469388,0.389716,0.877778,0.005398,0.52381,0.106595,0.253357,0.679825,0.0,0.0,0.0,0.253368,0.448980,0.048503,0.0,0.101945,0.160437,4234
1043,0.487179,0.469388,0.465494,0.255556,0.005398,0.52381,0.106595,0.253357,0.679825,0.0,0.0,0.0,0.253368,0.448980,0.048503,0.0,0.101945,0.173627,4241
1044,0.487179,0.469388,0.427605,0.566667,0.005398,0.52381,0.106595,0.253357,0.679825,0.0,0.0,0.0,0.253368,0.448980,0.048503,0.0,0.101945,0.172797,4247


In [55]:
import math
output = X[['id', 'pred']]
output = output.merge(prices, how = 'left', on = 'id')
#output['intermediary'] = math.exp(.087*(output['pred'] - 31.17))
output['percent_discount'] = 58.73/(1+np.exp(.087*(output['pred'] - 31.17)))
output['percent_discount'] = .01 * output['percent_discount']
output['new_price'] = output['price'] * (1 - output['percent_discount'])
output.to_csv('showing_discounted_prices.csv', index = False)